<a href="https://colab.research.google.com/github/vasist1987/HackerRank_Python/blob/master/Amazon_hackerEarth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon business analyst challenge

## importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics
!pip install --quiet optuna
import optuna
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
!pip install treeinterpreter
from sklearn.preprocessing import StandardScaler
!pip install lime
import lime
import lime.lime_tabular

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



### method to input data set

In [2]:
class dataImports():
    
    def __init__(self, df=None):
        self.df = df
        
    def openZipFileCsv(self,zipPath,fileName):
        zp=zipfile.ZipFile(zipPath,'r')
        self.df= pd.read_csv(zp.open(fileName))
        return self
        
    def import_Data_File(self,objectName,src_type,sheet=None):   
        if src_type=='csv':
            self.df=pd.read_csv(objectName)       
        if src_type=='excel':
            self.df=pd.read_excel(objectName, sheet_name = None)
        return self
    def import_Data_Database(self,src_type,tbname,server_name,db_name,uname,pwd):
        if src_type=='mssql':
            con=getConnectionMSSQL(server_name,db_name,uname,pwd)
            self.df=selectFromDb(tbname,server_name,db_name,uname,pwd,con)
        return self
        

In [3]:
train=(dataImports().import_Data_File('/content/Train.csv','csv')).df
test=(dataImports().import_Data_File('/content/Test.csv','csv')).df
sample_submission=(dataImports().import_Data_File('/content/sample submission.csv','csv')).df

### initial data analysis

In [4]:
train.head()

,EmpID,EmpName,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,YearsOfExperince,GraduationYear,CurrentCTC,ExpectedCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken,BiasInfluentialFactor,FitmentPercent
0,11041,John,English,35,Male,JR85289,B.Tech,Electrical,Tier 1,7,12,2009,21,26,Married,5,SSE,Enterprise,Design,20,YearsOfExperince,95.40
1,15079,William,English,26,Male,JR87525,B.Tech,Artificial Intelligence,Tier 3,7,3,2018,15,19,Married,5,BA,MidSized,Engineering,6,NaN,67.09
2,18638,James,English,36,Female,JR87525,PhD,Computer Science,Tier 1,6,6,2015,15,24,Single,5,SDE,MidSized,Engineering,19,Gender,91.26
3,3941,Charles,English,29,Female,JR87525,BCA,Information Technology,Tier 2,5,6,2015,16,24,Married,5,SDE,Startup,Product,16,Gender,72.29
4,5936,George,English,25,Male,JR70175,Dual M.Tech,Computer Science,Tier 3,8,2,2019,24,32,Married,5,SDE,Enterprise,Engineering,10,DegreeBranch,86.34


In [5]:
test.head()

,EmpID,EmpName,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,YearsOfExperince,GraduationYear,CurrentCTC,ExpectedCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken
0,5664,Theo,Native,35,Male,JR85289,PhD,Information Technology,Tier 2,5,5,2016,27,34,Single,4,SDE,Enterprise,Finance,18
1,23568,Vernie,English,35,Male,JR85289,B.Tech,Information Technology,Tier 3,9,12,2009,29,33,Married,4,EM,Enterprise,Engineering,8
2,21490,Wendell,English,28,Male,JR87525,M.Tech,Artificial Intelligence,Tier 2,9,2,2019,18,25,Married,3,SDE,Startup,Engineering,18
3,8363,West,Hindi,37,Male,JR87525,MS,Computer Science,Tier 2,8,11,2010,7,12,Single,5,DS,Enterprise,Product,17
4,6165,Alford,Hindi,36,Female,JR85289,PhD,Artificial Intelligence,Tier 2,7,6,2015,22,32,Married,4,SDE,Startup,Engineering,9


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13645 entries, 0 to 13644
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmpID                    13645 non-null  int64  
 1   EmpName                  13645 non-null  object 
 2   LanguageOfCommunication  13645 non-null  object 
 3   Age                      13645 non-null  int64  
 4   Gender                   13645 non-null  object 
 5   JobProfileIDApplyingFor  13645 non-null  object 
 6   HighestDegree            13645 non-null  object 
 7   DegreeBranch             13645 non-null  object 
 8   GraduatingInstitute      13645 non-null  object 
 9   LatestDegreeCGPA         13645 non-null  int64  
 10  YearsOfExperince         13645 non-null  int64  
 11  GraduationYear           13645 non-null  int64  
 12  CurrentCTC               13645 non-null  int64  
 13  ExpectedCTC              13645 non-null  int64  
 14  MartialStatus         

In [7]:
train.BiasInfluentialFactor.value_counts()

DegreeBranch          2858
Gender                2605
YearsOfExperince      1561
CurrentCompanyType    1174
EmpScore               659
HighestDegree          582
Ethinicity             328
MartialStatus          280
LatestDegreeCGPA       262
Name: BiasInfluentialFactor, dtype: int64

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmpID                    8745 non-null   int64 
 1   EmpName                  8745 non-null   object
 2   LanguageOfCommunication  8745 non-null   object
 3   Age                      8745 non-null   int64 
 4   Gender                   8745 non-null   object
 5   JobProfileIDApplyingFor  8745 non-null   object
 6   HighestDegree            8745 non-null   object
 7   DegreeBranch             8745 non-null   object
 8   GraduatingInstitute      8745 non-null   object
 9   LatestDegreeCGPA         8745 non-null   int64 
 10  YearsOfExperince         8745 non-null   int64 
 11  GraduationYear           8745 non-null   int64 
 12  CurrentCTC               8745 non-null   int64 
 13  ExpectedCTC              8745 non-null   int64 
 14  MartialStatus            8745 non-null  

### encoding categorical variables

In [9]:
objList = [ 'LanguageOfCommunication',  'Gender',  'HighestDegree', 'DegreeBranch', 'GraduatingInstitute',  'MartialStatus', 'CurrentDesignation', 'CurrentCompanyType','DepartmentInCompany']
lbe = LabelEncoder()
for feat in objList:
    lbe.fit(train[feat].astype('str').values)
    diz_map_train = dict(zip(lbe.classes_, lbe.transform(lbe.classes_)+1))

    for i in set(test[feat].astype('str')).difference(train[feat].astype('str')):
        diz_map_train[i] = 0

    train[feat] = [diz_map_train[i] for i in train[feat].astype('str').values]
    test[feat] = [diz_map_train[i] for i in test[feat].astype('str').values]

In [10]:
# dropping columns not required for analysis
df_train=train.drop(['EmpID','EmpName','JobProfileIDApplyingFor'],axis=1)
df_test=test.drop(['EmpID','EmpName','JobProfileIDApplyingFor'],axis=1)

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8745 entries, 0 to 8744
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   LanguageOfCommunication  8745 non-null   int64
 1   Age                      8745 non-null   int64
 2   Gender                   8745 non-null   int64
 3   HighestDegree            8745 non-null   int64
 4   DegreeBranch             8745 non-null   int64
 5   GraduatingInstitute      8745 non-null   int64
 6   LatestDegreeCGPA         8745 non-null   int64
 7   YearsOfExperince         8745 non-null   int64
 8   GraduationYear           8745 non-null   int64
 9   CurrentCTC               8745 non-null   int64
 10  ExpectedCTC              8745 non-null   int64
 11  MartialStatus            8745 non-null   int64
 12  EmpScore                 8745 non-null   int64
 13  CurrentDesignation       8745 non-null   int64
 14  CurrentCompanyType       8745 non-null   int64
 15  Depa

### scaling input features

In [12]:

cl=[df_test.columns]
for i in cl:
  norm = StandardScaler().fit(df_train[i])

  # transform training data
  df_train[i] = norm.transform(df_train[i])

  # transform testing dataabs
  df_test[i] = norm.transform(df_test[i])
X=df_train[df_test.columns]
y=df_train.FitmentPercent

In [13]:
# Creating Train-Test Split for validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Initial model with Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_train)
from sklearn.metrics import mean_squared_error,r2_score
mse = mean_squared_error(y_train, y_pred)**(0.5)
r2=r2_score(y_train, y_pred)
print(r2,mse)
y_pred = regressor.predict(X_test)
from sklearn.metrics import mean_squared_error,r2_score
mse = mean_squared_error(y_test, y_pred)**(0.5)
r2=r2_score(y_test, y_pred)
print(r2,mse)

0.2735277645015861 12.724325231886352
0.26325784447739586 12.662222550601506


### initial Random forest model
### Both are providing same results , these results are without any feature engineering or hyper parameter tuning


In [15]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
from sklearn.metrics import mean_squared_error,r2_score
mse = mean_squared_error(y_train, y_pred)**(0.5)
r2=r2_score(y_train, y_pred)
print(r2,mse)
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error,r2_score
mse = mean_squared_error(y_test, y_pred)**(0.5)
r2=r2_score(y_test, y_pred)
print(r2,mse)

0.6613134531206346 8.688088250669358
0.6342476503662704 8.921661563378022


## Tree Interpretor to find influencing factor

In [39]:
from sklearn.model_selection import GridSearchCV
param_grid = {"max_depth": [5, 20, 15, 20],"max_features": [5, 10,15,18],"min_samples_leaf": [3, 4, 5],"min_samples_split": [8, 10, 12],"n_estimators": [20, 100, 150]}

rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 

                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

grid_search.best_params_

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 393 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 722 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 1133 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed: 15.2min finished


{'max_depth': 5,
 'max_features': 15,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 100}

In [40]:
from treeinterpreter import treeinterpreter as ti
# fit a scikit-learn's regressor model
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features=15, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)
rf.fit(X_train, y_train)

prediction, bias, contributions = ti.predict(rf, X_train)
mse = mean_squared_error(y_train, prediction)**(0.5)
r2=r2_score(y_train, prediction)
print(r2,mse)
prediction, bias, contributions = ti.predict(rf, X_test)
mse = mean_squared_error(y_test, prediction)**(0.5)
r2=r2_score(y_test, prediction)
print(r2,mse)

0.6524950878975123 8.800467116978169
0.6356660325261525 8.904345724152119


In [17]:
prediction, bias, contributions = ti.predict(rf, df_test)
influence_list=[]
cols=X_train.columns
for i in range(len(contributions)):
  influence_list.append(cols[np.argmax(contributions[i], axis=0)])
influence=pd.DataFrame(influence_list)
pred=pd.DataFrame(prediction)
empId=pd.DataFrame(test.EmpID)
sub=pd.concat([empId,influence,pred],axis=1)
sub.columns=['EmpID','BiasInfluentialFactor','FitmentPercent']
sub.to_csv('ams_Submission_3.csv',index=False)
from google.colab import files
files.download('ams_Submission_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# r2 is around 65 percent it requires better feature engineering to further improve but the score is close to 100 percent in the submission due to lack of time submitting this solution as is.

In [18]:
# explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names = X_test.columns,
#                                                    class_names=['FitmentPercent'],verbose=True, mode='regression')

In [19]:
# j = 392
# exp = explainer.explain_instance(X_test.values[j], model.predict, num_features=25)
# exp.show_in_notebook()

In [20]:
# exp.local_exp[1]